In [2]:
"""
	Autores:
	- Astrid Alvarado
	- Junior Lara
"""

# Asegurese de seleccionar el Kernel de Python 3, puede depender de su entorno.

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round, count, when, first, explode, split

In [3]:
spark = SparkSession\
    .builder\
    .appName("Solver problema 3 - DF")\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/01 15:05:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# ITEM 1: Ver el comportamiento de la popularidad en función de la distribución.
# Archivo necesiario: final_animedataset.csv
# Campos necesarios:
# 	- title
# 	- gender
# 	- popularity

# Direccionio del archivo
# Ej. file = "../../tests/vshort-final_animedataset.csv"
file = "../../official/final_animedataset.csv"

final_animedataset = spark\
	.read\
	.format("csv")\
	.option("header", "true")\
	.load(file)

final_animedataset.show(20)

+--------+--------+--------+-------+------+--------------------+----+------------+-----+---------+------+----------+--------------------+
|username|anime_id|my_score|user_id|gender|               title|type|      source|score|scored_by|  rank|popularity|               genre|
+--------+--------+--------+-------+------+--------------------+----+------------+-----+---------+------+----------+--------------------+
|karthiga|      21|       9|2255153|Female|           One Piece|  TV|       Manga| 8.54|   423868|  91.0|        35|Action, Adventure...|
|karthiga|      59|       7|2255153|Female|             Chobits|  TV|       Manga| 7.53|   175388|1546.0|       188|Sci-Fi, Comedy, D...|
|karthiga|      74|       7|2255153|Female|        Gakuen Alice|  TV|       Manga| 7.77|    33244| 941.0|      1291|Comedy, School, S...|
|karthiga|     120|       7|2255153|Female|       Fruits Basket|  TV|       Manga| 7.77|   167968| 939.0|       222|Slice of Life, Co...|
|karthiga|     178|       7|225515

In [5]:
# Calcular la cantidad de usuarios por género y la popularidad para cada título
count_female_male_by_title = final_animedataset.groupBy("title") \
           .agg(
               count(when(col("gender") == "Female", 1)).alias("female_count"),  # Contar las females
               count(when(col("gender") == "Male", 1)).alias("male_count"),      # Contar los males
               first("popularity").alias("popularity")                          # Tomar la popularidad única
           )

# Agregar las columnas de porcentaje para hombres y mujeres
count_female_male_by_title_with_percen = count_female_male_by_title.withColumn(
    "female_percentage", round((col("female_count") / (col("female_count") + col("male_count")) * 100), 2)
).withColumn(
    "male_percentage", round((col("male_count") / (col("female_count") + col("male_count")) * 100), 2)
)

# Mostrar los resultados
count_female_male_by_title_with_percen.show(20)

+--------------------+------------+----------+----------+-----------------+---------------+
|               title|female_count|male_count|popularity|female_percentage|male_percentage|
+--------------------+------------+----------+----------+-----------------+---------------+
|             """0"""|         182|       308|      6833|            37.14|          62.86|
|"""Bungaku Shoujo...|        1418|      2363|      2978|             37.5|           62.5|
|"""Bungaku Shoujo...|        1621|      2799|      2581|            36.67|          63.33|
|"Boku no Imouto w...|         683|      2123|      3073|            24.34|          75.66|
|"Boku no Imouto w...|         119|       425|      6115|            21.88|          78.13|
|"Gyakuten Saiban:...|        2172|      5519|    5839.0|            28.24|          71.76|
|"Ikoku Meiro no C...|         549|      1002|      4334|             35.4|           64.6|
|"Mogyutto ""Love"...|         642|      1233|      3848|            34.24|     

In [6]:
# Exportar los resultados a un archivo CSV "count_female_male_by_title_with_percen"
count_female_male_by_title_with_percen.write.csv("count_female_male_by_title_with_percen", header=True, mode="overwrite")

In [7]:
# ITEM 2: Ver la tendencia de géneros de anime en función de la distribución.
# Archivo necesiario: final_animedataset.csv
# Campos necesarios:
# 	- gender
# 	- genre

# Separar los géneros en la columna "genre" y explotar la lista en filas individuales
exploded_genres = final_animedataset.withColumn("genre", explode(split(col("genre"), ", ")))

# Contar la cantidad de usuarios por género (Female y Male) para cada género de anime
count_female_male_by_genres = exploded_genres.groupBy("genre") \
           .agg(
               count(when(col("gender") == "Female", 1)).alias("female_count"),  # Contar las females
               count(when(col("gender") == "Male", 1)).alias("male_count")       # Contar los males
           )

# Agregar las columnas de porcentaje para hombres y mujeres con redondeo a 2 decimales
count_female_male_by_genres_with_percen = count_female_male_by_genres.withColumn(
    "female_percentage", round((col("female_count") / (col("female_count") + col("male_count")) * 100), 2)
).withColumn(
    "male_percentage", round((col("male_count") / (col("female_count") + col("male_count")) * 100), 2)
)

# Mostrar los resultados
count_female_male_by_genres_with_percen.show(20)

+------------+------------+----------+-----------------+---------------+
|       genre|female_count|male_count|female_percentage|male_percentage|
+------------+------------+----------+-----------------+---------------+
|      Seinen|      755783|   2401731|            23.94|          76.06|
|     Romance|     3058903|   7422955|            29.18|          70.82|
|    Dementia|      123962|    290713|            29.89|          70.11|
|       Magic|     1044607|   2571788|            28.89|          71.11|
|    Thriller|      351992|    903155|            28.04|          71.96|
|       Josei|      223277|    253220|            46.86|          53.14|
|   Adventure|     1970463|   5195149|             27.5|           72.5|
| Super Power|      810990|   2386774|            25.36|          74.64|
|Martial Arts|      242263|    845883|            22.26|          77.74|
|      Sports|      355357|    847317|            29.55|          70.45|
|      Shoujo|     1115893|    965196|            5

In [8]:
# Exportar los resultados a un archivo CSV "count_female_male_by_genres_with_percen"
count_female_male_by_genres_with_percen.write.csv("count_female_male_by_genres_with_percen", header=True, mode="overwrite")

In [9]:
# ITEM 3: Ver si existe relación acerca de las preferencias de la distribución con respecto al origen (source) de los animes que consumen.
# Archivo necesiario: final_animedataset.csv
# Campos necesarios:
# 	- source
# 	- gender

# Calcular la cantidad de hombres y mujeres por origen (source) de los animes
count_female_male_by_source = final_animedataset.groupBy("source") \
           .agg(
               count(when(col("gender") == "Female", 1)).alias("female_count"),  # Contar las females
               count(when(col("gender") == "Male", 1)).alias("male_count")      # Contar los males
           )

# Agregar las columnas de porcentaje para hombres y mujeres con redondeo a 2 decimales
count_female_male_by_source_with_percen = count_female_male_by_source.withColumn(\
	"female_percentage", round((col("female_count") / (col("female_count") + col("male_count")) * 100), 2)
).withColumn(
    "male_percentage", round((col("male_count") / (col("female_count") + col("male_count")) * 100), 2)
)

# Mostrar los resultados
count_female_male_by_source_with_percen.show(20)


+-------------+------------+----------+-----------------+---------------+
|       source|female_count|male_count|female_percentage|male_percentage|
+-------------+------------+----------+-----------------+---------------+
|           TV|        2172|      5519|            28.24|          71.76|
| Visual novel|      646657|   1800061|            26.43|          73.57|
|Digital manga|        6128|     11239|            35.29|          64.71|
|     Original|     1778273|   4574674|            27.99|          72.01|
|        Novel|      410685|    899754|            31.34|          68.66|
| Picture book|        5270|      7110|            42.57|          57.43|
|         Book|       28696|     39683|            41.97|          58.03|
|      Unknown|      381452|   1011208|            27.39|          72.61|
|        Other|      115514|    294737|            28.16|          71.84|
|        Radio|         902|      1939|            31.75|          68.25|
|        Manga|     4365668|  10194460

In [10]:
# Exportar los resultados a un archivo CSV "count_female_male_by_source_with_percen"
count_female_male_by_source_with_percen.write.csv("count_female_male_by_source_with_percen", header=True, mode="overwrite")

In [8]:
# ITEM 4: Ver si es posible detectar un patrón referente a qué estudio es más propenso a sacar los tipos de animes que están en estas características.
# Archivo necesiario: 
# Campos necesarios:
# - 



In [11]:
spark.stop()